# Initializing the SysML v2 API

In [1]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json
from datetime import datetime

host = "<specify protocol://host:port of the server that is a provider of the SysML v2 REST/HTTP API"

# Get projects

In [2]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)

if projects_response.status_code == 200:
    projects = projects_response.json()
    projects_data = list(map(lambda b: {'Project Name':b['name'], 'Project ID':b['@id']}, projects))
    df = pd.DataFrame.from_records(projects_data).sort_values(by='Project Name').style.hide(axis='index')
    display(df)
else:
    pprint("Problem in fetching projects")

Project Name,Project ID
10b-Trade-off Among Alternative Configurations Tue May 17 15:07:31 EDT 2022,862bf932-940e-4707-af0a-6dfda5e0e24e
10c-Fuel Economy Analysis Tue May 17 15:07:51 EDT 2022,afe315eb-87f7-4ae1-8e64-2305d4e2d725
10d-Dynamics Analysis Tue May 17 15:08:17 EDT 2022,0b42e175-272e-481d-aaea-d14fbf1b8e03
11a-View-Viewpoint Tue May 17 15:08:41 EDT 2022,c22c4769-979e-4590-a4e5-a505884ca136
11b-Safety and Security Feature Views Tue May 17 15:08:59 EDT 2022,05e04320-f883-46a7-8e86-e9b27dd38bbf
12a-Dependency Tue May 17 15:09:17 EDT 2022,18d6e46e-0fab-4bb8-9c34-5cca040b4317
12b-Allocation Tue May 17 15:09:29 EDT 2022,a60d254b-322d-46ad-9dce-cda7b63b3f66
13a-Model Containment Tue May 17 15:09:42 EDT 2022,3ed5fa65-3098-4d0f-a061-fb986e3fc204
13b-Safety and Security Features Element Group Tue May 17 15:09:55 EDT 2022,0ad176f2-ded0-4802-a41f-acf44fd63202
13b-Safety and Security Features Element Group-1 Tue May 17 15:10:08 EDT 2022,20291d54-1092-4dba-9769-a9282f70371c


# Create a new project

In [3]:
timestamp = datetime.now()
project_name = f"Spacecraft {timestamp}"
project_data = {
  "@type":"Project",
  "name": project_name,
  "description": "Spacecraft project with data"
}

project_post_url = f"{host}/projects" 

project_post_response = requests.post(project_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(project_data))

project_id = ""

if project_post_response.status_code == 200:
    project_response_json = project_post_response.json()
    pprint(project_response_json)
    project_id = project_response_json['@id']
    project_name = project_response_json['name']
else:
    pprint(f"Problem in creating a new Spacecraft project at {timestamp}")
    pprint(project_post_response)

{'@id': '25021e68-5a8a-426e-a43f-e8b15610519c',
 '@type': 'Project',
 'defaultBranch': {'@id': '813b2e97-82b5-4930-8100-0881d5658a17'},
 'description': 'Spacecraft project with data',
 'name': 'Spacecraft 2022-06-07 11:54:09.837268'}


# Get branches

In [4]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches of Spacecraft project {project_id}")
    pprint(branches_response)

Branch Name,Branch ID,Ref Commit (head)
main,813b2e97-82b5-4930-8100-0881d5658a17,None


# Get tags

In [5]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    pprint(tags)
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags of Spacecraft project {project_id}")
    pprint(tags_response)

[]


# Create 1st Commit

In [6]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Spacecraft System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Payload System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Propulsion System"
      }
    }
  ]
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit1_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit1_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '1da6d589-8232-47c5-8e5b-d8c1ced0961d',
 '@type': 'Commit',
 'owningProject': {'@id': '25021e68-5a8a-426e-a43f-e8b15610519c'},
 'previousCommit': None,
 'timestamp': '2022-06-07T11:54:16.923686-04:00'}


In [7]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit1_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit1_id}.")
    pprint(element_get_response)

Element Name,Element ID
Payload System,3bacaac2-1548-498c-a3a9-35e1224fb333
Propulsion System,40c690e7-fc67-4e0b-9de1-51f5fa63aaad
Spacecraft System,08f2762c-daee-4170-9d3a-1ea874796bf0


# Create 2nd Commit

In [8]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Avionics System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Power System"
      }
    }
  ],
  "previousCommit": {
      "@id": commit1_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit2_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit2_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4',
 '@type': 'Commit',
 'owningProject': {'@id': '25021e68-5a8a-426e-a43f-e8b15610519c'},
 'previousCommit': {'@id': '1da6d589-8232-47c5-8e5b-d8c1ced0961d'},
 'timestamp': '2022-06-07T11:54:21.512896-04:00'}


# Get all elements after the 2nd commit

In [9]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit2_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}.")
    pprint(element_get_response)

Element Name,Element ID
Avionics System,c3a9cd95-d892-4f80-8e74-e69279ace767
Payload System,3bacaac2-1548-498c-a3a9-35e1224fb333
Power System,67409bf0-066a-463d-98af-0bd29753daf3
Propulsion System,40c690e7-fc67-4e0b-9de1-51f5fa63aaad
Spacecraft System,08f2762c-daee-4170-9d3a-1ea874796bf0


# Create a tag

In [10]:
tag_body = {
    "@type": "Tag",
    "name": "Spacecraft Internal Release 0.1",
    "taggedCommit": {
      "@id": commit2_id
    }
}

tag_post_url = f"{host}/projects/{project_id}/tags" 

tag_post_response = requests.post(tag_post_url, 
                                  headers={"Content-Type": "application/json"}, 
                                  data=json.dumps(tag_body))

tag1_id = ""

if tag_post_response.status_code == 200:
    tag_response_json = tag_post_response.json()
    pprint(tag_response_json)
    tag1_id = tag_response_json['@id']
else:
    pprint(f"Problem in creating a new tag in Spacecraft project {project_id}.")
    pprint(tag_post_response)

{'@id': '24c65293-1361-473a-bd2b-cfe63843cb21',
 '@type': 'Tag',
 'name': 'Spacecraft Internal Release 0.1',
 'owningProject': {'@id': '25021e68-5a8a-426e-a43f-e8b15610519c'},
 'referencedCommit': {'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'},
 'taggedCommit': {'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'},
 'timestamp': '2022-06-07T11:54:25.857998-04:00'}


# Get Tags

In [11]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags in Spacecraft project {project_id}.")
    pprint(tag_post_response)

Tag Name,Tag ID,Tagged Commit
Spacecraft Internal Release 0.1,24c65293-1361-473a-bd2b-cfe63843cb21,{'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'}


# Create a new branch at the second commit (also tagged as 0.1)

In [12]:
branch_body = {
  "@type": "Branch",
  "name": "develop",
  "head": {
      "@id": commit2_id
    }
}

branch_post_url = f"{host}/projects/{project_id}/branches" 

branch_post_response = requests.post(branch_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(branch_body))

branch_develop_id = ""

if branch_post_response.status_code == 200:
    branch_response_json = branch_post_response.json()
    pprint(branch_response_json)
    branch_develop_id = branch_response_json['@id']
else:
    pprint(f"Problem in creating a new branch in Spacecraft project {project_id}")
    pprint(branch_post_response)

{'@id': '3bc848fe-5d2c-49c7-a91b-e4335edd3e7a',
 '@type': 'Branch',
 'head': {'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'},
 'name': 'develop',
 'owningProject': {'@id': '25021e68-5a8a-426e-a43f-e8b15610519c'},
 'referencedCommit': {'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'},
 'timestamp': '2022-06-07T11:54:30.360571-04:00'}


# Get branches

In [13]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches in Spacecraft project {project_id}")
    pprint(branches_response)

Branch Name,Branch ID,Ref Commit (head)
develop,3bc848fe-5d2c-49c7-a91b-e4335edd3e7a,{'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'}
main,813b2e97-82b5-4930-8100-0881d5658a17,{'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'}


# Create a new commit in the develop branch

In [14]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"GN & C System"
      }
    }
  ],
  "previousCommit": {
      "@id": commit2_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body),
                                      params={"branchId":branch_develop_id})

commit3_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit3_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '682a9635-10e4-45fb-80b8-299955d90f03',
 '@type': 'Commit',
 'owningProject': {'@id': '25021e68-5a8a-426e-a43f-e8b15610519c'},
 'previousCommit': {'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'},
 'timestamp': '2022-06-07T11:54:34.098474-04:00'}


# Get Branches

In [15]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches in Spacecraft project {project_id}")
    pprint(commit_post_response)

Branch Name,Branch ID,Ref Commit (head)
develop,3bc848fe-5d2c-49c7-a91b-e4335edd3e7a,{'@id': '682a9635-10e4-45fb-80b8-299955d90f03'}
main,813b2e97-82b5-4930-8100-0881d5658a17,{'@id': 'd2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'}


# Get all elements as of the latest commit on main branch

In [16]:
pprint(f"Fetching elements at commit {commit2_id}")
element_get_url = f"{host}/projects/{project_id}/commits/{commit2_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}")
    pprint(element_get_response)

'Fetching elements at commit d2b90e4b-e2fa-4603-b7ae-5ac1329fdfe4'


Element Name,Element ID
Avionics System,c3a9cd95-d892-4f80-8e74-e69279ace767
Payload System,3bacaac2-1548-498c-a3a9-35e1224fb333
Power System,67409bf0-066a-463d-98af-0bd29753daf3
Propulsion System,40c690e7-fc67-4e0b-9de1-51f5fa63aaad
Spacecraft System,08f2762c-daee-4170-9d3a-1ea874796bf0


# Get all elements as of the latest commit on develop branch

In [17]:
pprint(f"Fetching elements at commit {commit3_id}")
element_get_url = f"{host}/projects/{project_id}/commits/{commit3_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit3_id}")
    pprint(element_get_response)

'Fetching elements at commit 682a9635-10e4-45fb-80b8-299955d90f03'


Element Name,Element ID
Avionics System,c3a9cd95-d892-4f80-8e74-e69279ace767
GN & C System,27e47fbd-9939-46f5-b1de-32cc762acfa0
Payload System,3bacaac2-1548-498c-a3a9-35e1224fb333
Power System,67409bf0-066a-463d-98af-0bd29753daf3
Propulsion System,40c690e7-fc67-4e0b-9de1-51f5fa63aaad
Spacecraft System,08f2762c-daee-4170-9d3a-1ea874796bf0
